In [ ]:
from pathlib import Path
from pandas import Timestamp

import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
from IPython.display import Image
# Import seaborn
import seaborn as sns

import hillmaker as hm

In [ ]:
ssu_stopdata = '../data/ShortStay.csv'
ssu_stops_df = pd.read_csv(ssu_stopdata, parse_dates=['InRoomTS','OutRoomTS'])
ssu_stops_df.info() # Check out the structure of the resulting DataFrame

In [ ]:
# Required inputs
scenario_name = 'ssu_1'
stops_df = ssu_stops_df
in_field_name = 'InRoomTS'
out_field_name = 'OutRoomTS'
start_date = '1996-01-01'
end_date = '1996-09-30'

# Optional inputs

cat_field_name = 'PatType'
verbosity = 1 # INFO level logging
output_path = './output'
bin_size_minutes = 60

s1 = hm.Scenario(scenario_name=scenario_name, 
                         stops_df=stops_df,
                         in_field=in_field_name,
                         out_field=out_field_name,
                         start_analysis_dt=start_date,
                         end_analysis_dt=end_date,
                         cat_field=cat_field_name,
                         output_path=Path('./output'),
                         verbosity=verbosity)

In [ ]:
s1.make_hills()

In [ ]:
s1.get_summary_df?

In [ ]:
occ_summary_cat_df = s1.get_summary_df()
occ_summary_cat_df

In [ ]:
occ_summary_df = s1.get_summary_df(by_category=False)
occ_summary_df

In [ ]:
def make_week_hill_plot(summary_df: pd.DataFrame, scenario_name: str, metric: str,
                        

                        bin_size_minutes: int = 60,
                        cap: int = None,
                        first_dow: str = 'Mon',
                        xlabel: str = 'Hour',
                        ylabel: str = 'Patients',
                        export_path: Path | str | None = None,):
    """
    Makes and optionally exports week plot for occupancy, arrivals, or departures.

    Takes output DataFrames of `summarize.summarize` and plots mean and percentile
    values for occupancy, arrival, and departure categories.

    Parameters
    ----------
    summary_df : DataFrame
        Single summary df from the output of `summarize.summarize`
    scenario_name : str
        Used in output filenames
    metric : str
        Name of make_hills summary df being plotted
    export_path : str or Path, optional
        Destination path for exported png files, default is current directory
    bin_size_minutes : int, optional
        Number of minutes in each time bin of the day, default is 60. Use a value that
        divides into 1440 with no remainder
    cap : int, optional
        Capacity of area being analyzed, default is None
    week_range : str
        Week range of summary df. Default is 'week', can also take the form of
        the first three characters of a day of week name (ex: 'tue')
    xlabel : str
        x-axis label, default='Hour'
    ylabel : str
        y-axis label, default='Patients'
    export_png : bool, default is False
        If True, plot is exported to png file to `export_path`
    """

    plt.style.use('seaborn-darkgrid')
    fig1 = plt.figure(figsize=(15, 10))
    ax1 = fig1.add_subplot(1, 1, 1)

    # infer number of days being plotted
    num_days = len(summary_df) / (60 / bin_size_minutes * 24)

    # Create a list to use as the X-axis values
    num_bins = num_days * 1440 / bin_size_minutes
    # TODO: This is a Monday. Make flexible so any dow can be "first".
    base_date_for_first_dow = '2015-01-05'
    timestamps = pd.date_range(base_date_for_first_dow, periods=num_bins, freq=f'{bin_size_minutes}Min').tolist()

    # Choose appropriate major and minor tick locations
    major_tick_locations = pd.date_range(f'{base_date_for_first_dow} 12:00:00', periods=7, freq='24H').tolist()
    minor_tick_locations = pd.date_range(f'{base_date_for_first_dow} 06:00:00', periods=42, freq='4H').tolist()

    # Set the tick locations for the axes object
    ax1.set_xticks(major_tick_locations)
    ax1.set_xticks(minor_tick_locations, minor=True)

    # Specify the mean occupancy and percentile values. TODO - let user choose series to plot
    mean_occ = summary_df['mean']
    pctile_occ = summary_df['p95']

    # Styling of bars, lines, plot area
    # Style the bars for mean occupancy
    bar_color = 'steelblue'

    # Style the line for the occupancy percentile
    # pctile_line_style = '-'
    # pctile_color = 'grey'
    pctile_colors = ['blue', 'green']
    pctile_linestyles = ['-', '--']

    # Add data to the plot
    # Mean occupancy as bars - here's the GOTCHA involving the bar width
    bar_width = 1 / (1440 / bin_size_minutes)
    ax1.bar(timestamps, mean_occ, label=f'Mean {metric}', width=bar_width, color=bar_color)

    # Some percentile as a line
    ax1.plot(timestamps, pctile_occ, linestyle=pctile_line_style, label=f'95th %ile {metric}', color=pctile_color)

    # establish capacity horizontal line if supplied
    if cap is not None and metric == 'occupancy':
        plt.axhline(cap, color='r', linestyle='--', label='Capacity')

    # Create formatter variables
    day_fmt = '' if num_days == 1 else '%a'
    dayofweek_formatter = DateFormatter(day_fmt)
    qtrday_formatter = DateFormatter('%H')

    # Format the tick labels
    ax1.xaxis.set_major_formatter(dayofweek_formatter)
    ax1.xaxis.set_minor_formatter(qtrday_formatter)

    # Slide the major tick labels underneath the default location by 20 points
    ax1.tick_params(which='major', pad=20)

    # Add other chart elements

    # Set plot and axis titles
    sup_title = fig1.suptitle(f'{metric.title()} by Time of Day - {week_range.title()}\n{scenario_name.title()}',
                              x=0.125, y=0.95, horizontalalignment='left', verticalalignment='top', fontsize=16)

    ax1.set_title('All category types', loc='left', style='italic')
    ax1.set_xlabel(xlabel)
    ax1.set_ylabel(ylabel)

    # Legend
    ax1.legend(loc='best', frameon=True, facecolor='w')

    # save figure
    if export_path is not None:
        week_range_str = 'week'
        plot_png = f'{scenario_name}_{metric}_plot_{week_range_str}.png'
        png_wpath = Path(export_path, plot_png)
        plt.savefig(png_wpath, bbox_extra_artists=[sup_title], bbox_inches='tight')

    # Suppress plot output in notebook
    plt.close()

    return fig1

In [ ]:
pctile_colors = ['blue', 'green']
pctile_linestyles = ['-', '--']

In [ ]:
from cycler import cycler
cc = (cycler(color=pctile_colors) +
      cycler(linestyle=pctile_linestyles))
for d in cc:
    print(d)

In [ ]:
with plt.rc_context({'axes.prop_cycle': cc}):

## First dow

In [ ]:
def find_first_dow(year, month, dow):
    d = Timestamp(year, month, 1)
    offset = -d.weekday() #weekday = 0 means monday
    return d + timedelta(offset)

In [ ]:
year = 2015
month = 1
dow = 0

d = Timestamp(year, month, 1)
d

In [ ]:
dow_1 = d.weekday()
dow_1

In [ ]:
d + pd.Timedelta(7, unit="d") - pd.Timedelta(dow_1, unit="d")

In [ ]:
(d + pd.Timedelta(7, unit="d") - pd.Timedelta(dow_1, unit="d")).weekday()

In [ ]:
(d + pd.Timedelta(7, unit="d") - pd.Timedelta(dow_1, unit="d")).weekday()